У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [188]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import copy

In [25]:
df = pd.read_csv("data/customer_segmentation_train.csv",index_col='ID' )

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


In [17]:
df.describe()

,Age,Work_Experience,Family_Size
count,8068.000000,7239.000000,7733.000000
mean,43.466906,2.641663,2.850123
std,16.711696,3.406763,1.531413
min,18.000000,0.000000,1.000000
25%,30.000000,0.000000,2.000000
50%,40.000000,1.000000,3.000000
75%,53.000000,4.000000,4.000000
max,89.000000,14.000000,9.000000


In [16]:
df.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [29]:
df.isnull().sum()

Gender               0
Ever_Married         0
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [28]:
df.Ever_Married.value_counts()

Ever_Married
Yes        4643
No         3285
Unknown     140
Name: count, dtype: int64

In [27]:
df.Ever_Married.fillna("Unknown", inplace=True)

In [30]:
df.Graduated.value_counts()

Graduated
Yes    4968
No     3022
Name: count, dtype: int64

In [31]:
df.Graduated.fillna("Unknown", inplace=True)

In [32]:
df.Profession.value_counts()

Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64

In [33]:
df.Profession.fillna("Unknown", inplace=True)

In [49]:
df.Work_Experience.mean(), df.Work_Experience.median()

(2.641663213150988, 1.0)

In [56]:
df.Work_Experience.isna().sum()

829

In [53]:
df['Work_Experience_was_missed'] = df['Work_Experience'].isnull().astype('int')

In [54]:
df['Work_Experience_was_missed'].value_counts()

Work_Experience_was_missed
0    7239
1     829
Name: count, dtype: int64

In [58]:
df['Work_Experience'].fillna(df['Work_Experience'].median(), inplace=True)

In [65]:
df['Family_Size_was_missed'] = df['Family_Size'].isnull().astype('int')

In [66]:
df['Family_Size'].fillna(df['Family_Size'].median(), inplace=True)

In [67]:
df['Family_Size'].value_counts()

Family_Size
2.0    2390
3.0    1832
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      8068 non-null   object 
 1   Ever_Married                8068 non-null   object 
 2   Age                         8068 non-null   int64  
 3   Graduated                   8068 non-null   object 
 4   Profession                  8068 non-null   object 
 5   Work_Experience             8068 non-null   float64
 6   Spending_Score              8068 non-null   object 
 7   Family_Size                 8068 non-null   float64
 8   Var_1                       8068 non-null   object 
 9   Segmentation                8068 non-null   object 
 10  Work_Experience_was_missed  8068 non-null   int64  
 11  Family_Size_was_missed      8068 non-null   int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 819.4+ KB


In [72]:
df['Var_1'].fillna('Undefined', inplace=True)

In [75]:
df['Spending_Score'].value_counts()

Spending_Score
Low        4878
Average    1974
High       1216
Name: count, dtype: int64

In [76]:
number_columns = df.select_dtypes(include="number")
number_columns

,Age,Work_Experience,Family_Size,Work_Experience_was_missed,Family_Size_was_missed
ID,,,,,
462809,22,1.0,4.0,0,0
462643,38,1.0,3.0,1,0
466315,67,1.0,1.0,0,0
461735,67,0.0,2.0,0,0
462669,40,1.0,6.0,1,0
...,...,...,...,...,...
464018,22,0.0,7.0,0,0
464685,35,3.0,4.0,0,0
465406,33,1.0,1.0,0,0


In [77]:
category_columns = df.select_dtypes(include="object")
category_columns

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1,Segmentation
ID,,,,,,,
462809,Male,No,No,Healthcare,Low,Cat_4,D
462643,Female,Yes,Yes,Engineer,Average,Cat_4,A
466315,Female,Yes,Yes,Engineer,Low,Cat_6,B
461735,Male,Yes,Yes,Lawyer,High,Cat_6,B
462669,Female,Yes,Yes,Entertainment,High,Cat_6,A
...,...,...,...,...,...,...,...
464018,Male,No,No,Unknown,Low,Cat_1,D
464685,Male,No,No,Executive,Low,Cat_4,D
465406,Female,No,Yes,Healthcare,Low,Cat_6,D


In [83]:
Q1 = df.Family_Size.quantile(0.25)
Q2 = df.Family_Size.quantile(0.75)
IQR = Q2 - Q1
Q1 - 1.5 * IQR, Q2 + 1.5*IQR

(-1.0, 7.0)

In [84]:
df.Family_Size.value_counts()

Family_Size
2.0    2390
3.0    1832
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64

по цьому полю викидів є 94 кейси

In [86]:
corrdf = df.select_dtypes("number").corr()
corrdf

,Age,Work_Experience,Family_Size,Work_Experience_was_missed,Family_Size_was_missed
Age,1.000000,-0.177344,-0.273373,0.000560,-0.006819
Work_Experience,-0.177344,1.000000,-0.059692,-0.152667,0.001818
Family_Size,-0.273373,-0.059692,1.000000,0.012547,0.019940
Work_Experience_was_missed,0.000560,-0.152667,0.012547,1.000000,0.062570
Family_Size_was_missed,-0.006819,0.001818,0.019940,0.062570,1.000000


In [87]:
df.Segmentation.value_counts()

Segmentation
D    2268
A    1972
C    1970
B    1858
Name: count, dtype: int64

In [91]:
train_X, test_X, train_target, test_target = train_test_split(df.drop(columns="Segmentation"), df.Segmentation, test_size=0.4, random_state=42, stratify=df.Segmentation)

In [93]:
df.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,Work_Experience_was_missed,Family_Size_was_missed
ID,,,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D,0,0
462643,Female,Yes,38,Yes,Engineer,1.0,Average,3.0,Cat_4,A,1,0
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B,0,0
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B,0,0
462669,Female,Yes,40,Yes,Entertainment,1.0,High,6.0,Cat_6,A,1,0


In [95]:
df.Spending_Score.value_counts()

Spending_Score
Low        4878
Average    1974
High       1216
Name: count, dtype: int64

In [108]:
category_columns_list = category_columns.drop(columns=['Spending_Score', 'Segmentation']).columns.to_list()
print(category_columns_list)
category_columns_list2 = ['Spending_Score']
print(category_columns_list2)
number_columns_list = number_columns.columns.to_list()
print(number_columns_list)


['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1']
['Spending_Score']
['Age', 'Work_Experience', 'Family_Size', 'Work_Experience_was_missed', 'Family_Size_was_missed']


In [126]:
ordinal_categories = ['Spending_Score']
numeric_transformer = Pipeline(
    steps=[
        ('scaler', MinMaxScaler())
    ]
)
categorical_transfromer = Pipeline(
    steps=[
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
    ]
)

categorical_transfromer2 = Pipeline(
    steps=[
        ('ordinary', OrdinalEncoder(categories=[['Low', 'Average', 'High']]))
    ]
)

categorical_transfromer3 = Pipeline(
    steps=[
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, number_columns_list),
        ('cat', categorical_transfromer, category_columns_list),
        ('cat2', categorical_transfromer2, category_columns_list2)
    ]
)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [127]:
model = Pipeline(
    steps=[
        ('preproc', preprocessor),
        ('classifier', LogisticRegression(solver='liblinear'))
    ]
)

In [128]:
model.fit(train_X, train_target)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age', 'Work_Experience',
                                                   'Family_Size',
                                                   'Work_Experience_was_missed',
                                                   'Family_Size_was_missed']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Gender', 'Ever_Married',
                                                   'Graduated', 'Profession',
                                                   'Var_1']),
                                                 ('cat2',
                                                  Pipeline(steps=[('ordinary',
                                                                   OrdinalEncoder(categories=[['Low',
                                                                                               'Average',
                                                                                               'High']]))]),
                                                  ['Spending_Score'])])),
                ('classifier', LogisticRegression(solver='liblinear'))])

In [135]:
y_test_pred = model.predict(test_X)
y_train_pred = model.predict(train_X)
print(classification_report(train_target, y_train_pred))
print(classification_report(test_target, y_test_pred))

              precision    recall  f1-score   support

           A       0.43      0.45      0.44       789
           B       0.39      0.15      0.22       743
           C       0.46      0.63      0.53       788
           D       0.65      0.74      0.69       908

    accuracy                           0.51      3228
   macro avg       0.48      0.49      0.47      3228
weighted avg       0.49      0.51      0.48      3228

              precision    recall  f1-score   support

           A       0.43      0.46      0.44      1183
           B       0.39      0.15      0.22      1115
           C       0.48      0.66      0.56      1182
           D       0.63      0.71      0.67      1360

    accuracy                           0.51      4840
   macro avg       0.48      0.49      0.47      4840
weighted avg       0.49      0.51      0.48      4840



Без використання  самплінгу маємо не дуже хороші результати

In [140]:
train_X

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Work_Experience_was_missed,Family_Size_was_missed
ID,,,,,,,,,,,
464383,Female,Yes,81,Yes,Lawyer,1.0,Low,1.0,Cat_6,0,0
462661,Female,Yes,62,Yes,Artist,2.0,High,3.0,Cat_6,0,0
462762,Male,Yes,48,Yes,Artist,0.0,High,4.0,Cat_6,0,0
459463,Female,Yes,62,Yes,Artist,1.0,Average,3.0,Cat_6,1,0
467302,Male,No,26,Yes,Executive,9.0,Low,4.0,Cat_6,0,0
...,...,...,...,...,...,...,...,...,...,...,...
465209,Male,Yes,47,Yes,Artist,0.0,Average,2.0,Cat_3,0,0
461652,Female,Yes,53,Yes,Artist,1.0,Average,3.0,Cat_6,0,0
464475,Male,No,25,Yes,Healthcare,0.0,Low,4.0,Cat_6,0,0


In [141]:
smote = SMOTENC(categorical_features=[0,1,3,4,6,8])
train_X_smote, train_target_smote = smote.fit_resample(train_X, train_target)

In [146]:
def getPredictAndMetrics(model_object, train_input, train_target, test_input, test_target):
    model_object.fit(train_input, train_target)
    train_predicted = model_object.predict(train_input) 
    test_predicted = model_object.predict(test_input) 
    print("Train metrics") 
    print(classification_report(train_target, train_predicted))
    print("Test metrics") 
    print(classification_report(test_target, test_predicted))

In [147]:
model_smote = copy.deepcopy(model)
getPredictAndMetrics(model_smote, train_X_smote, train_target_smote, test_X, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.44      0.50      0.47      1360
           B       0.41      0.24      0.30      1360
           C       0.50      0.61      0.55      1360
           D       0.62      0.66      0.64      1360

    accuracy                           0.50      5440
   macro avg       0.49      0.50      0.49      5440
weighted avg       0.49      0.50      0.49      5440

Test metrics
              precision    recall  f1-score   support

           A       0.41      0.47      0.44       789
           B       0.38      0.25      0.30       743
           C       0.48      0.56      0.52       788
           D       0.67      0.69      0.68       908

    accuracy                           0.50      3228
   macro avg       0.49      0.49      0.48      3228
weighted avg       0.49      0.50      0.49      3228



/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [159]:
preprocessor_model = Pipeline(
    steps=[
        ('preprocessor', preprocessor)
    ]
)
train_x_encoded = preprocessor_model.fit_transform(train_X, train_target)


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [162]:
test_x_encoded = preprocessor_model.fit_transform(test_X, test_target)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [203]:
model_smote_tomek = copy.deepcopy(model)
smote_tomek = SMOTETomek()
train_x_smote_tomek, train_target_smote_tomek = smote_tomek.fit_resample(X=train_x_encoded, y=train_target)
model_smote_tomek = LogisticRegression(solver='liblinear')
getPredictAndMetrics(model_smote_tomek, train_x_smote_tomek, train_target_smote_tomek, test_x_encoded, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.46      0.49      0.47      1010
           B       0.41      0.23      0.29      1038
           C       0.52      0.67      0.59      1089
           D       0.67      0.72      0.69      1085

    accuracy                           0.53      4222
   macro avg       0.51      0.53      0.51      4222
weighted avg       0.52      0.53      0.51      4222

Test metrics
              precision    recall  f1-score   support

           A       0.40      0.44      0.42       789
           B       0.36      0.22      0.28       743
           C       0.47      0.60      0.53       788
           D       0.68      0.68      0.68       908

    accuracy                           0.50      3228
   macro avg       0.48      0.49      0.48      3228
weighted avg       0.49      0.50      0.49      3228



In [187]:
model_svc = SVC()
getPredictAndMetrics(model_svc, train_x_smote_tomek, train_target_smote_tomek, test_x_encoded, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.57      0.58      0.57      1012
           B       0.51      0.43      0.46      1044
           C       0.61      0.68      0.65      1089
           D       0.73      0.74      0.74      1093

    accuracy                           0.61      4238
   macro avg       0.60      0.61      0.60      4238
weighted avg       0.61      0.61      0.61      4238

Test metrics
              precision    recall  f1-score   support

           A       0.40      0.41      0.41       789
           B       0.37      0.33      0.35       743
           C       0.53      0.59      0.56       788
           D       0.68      0.67      0.67       908

    accuracy                           0.51      3228
   macro avg       0.50      0.50      0.50      3228
weighted avg       0.50      0.51      0.51      3228



In [170]:
model_svc_2 = Pipeline(
    steps=[
        ('preproc', preprocessor),
        ('classifier', SVC())
    ]
)

In [171]:
test_X_smote, test_target_smote = smote.fit_resample(test_X, test_target)

In [210]:
model_svc_2.fit(train_X_smote, train_target_smote)
getPredictAndMetrics(model_svc_2, train_X_smote, train_target_smote, test_X, test_target)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Train metrics
              precision    recall  f1-score   support

           A       0.50      0.58      0.54      1360
           B       0.48      0.39      0.43      1360
           C       0.60      0.60      0.60      1360
           D       0.66      0.68      0.67      1360

    accuracy                           0.56      5440
   macro avg       0.56      0.56      0.56      5440
weighted avg       0.56      0.56      0.56      5440

Test metrics
              precision    recall  f1-score   support

           A       0.41      0.47      0.44       789
           B       0.39      0.33      0.36       743
           C       0.55      0.55      0.55       788
           D       0.67      0.69      0.68       908

    accuracy                           0.52      3228
   macro avg       0.51      0.51      0.51      3228
weighted avg       0.52      0.52      0.52      3228



**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [191]:
logReg = LogisticRegression(solver="liblinear")
ovr_model = OneVsRestClassifier(logReg)
ovr_model.fit(train_x_encoded, train_target)
getPredictAndMetrics(ovr_model, train_x_encoded, train_target, test_x_encoded, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.43      0.46      0.44      1183
           B       0.39      0.15      0.22      1115
           C       0.48      0.66      0.56      1182
           D       0.63      0.71      0.67      1360

    accuracy                           0.51      4840
   macro avg       0.48      0.49      0.47      4840
weighted avg       0.49      0.51      0.48      4840

Test metrics
              precision    recall  f1-score   support

           A       0.43      0.45      0.44       789
           B       0.39      0.15      0.22       743
           C       0.46      0.63      0.53       788
           D       0.65      0.74      0.69       908

    accuracy                           0.51      3228
   macro avg       0.48      0.49      0.47      3228
weighted avg       0.49      0.51      0.48      3228



In [211]:
train_X_smote_encoded = preprocessor_model.fit_transform(train_X_smote, train_target)
test_X_smote_encoded = preprocessor_model.fit_transform(test_X_smote, test_target)
ovr_model2 = OneVsRestClassifier(logReg)
ovr_model2.fit(train_X_smote_encoded, train_target_smote)
getPredictAndMetrics(ovr_model2, train_X_smote_encoded, train_target_smote, test_x_encoded, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.44      0.50      0.47      1360
           B       0.41      0.24      0.30      1360
           C       0.50      0.61      0.55      1360
           D       0.62      0.66      0.64      1360

    accuracy                           0.50      5440
   macro avg       0.49      0.50      0.49      5440
weighted avg       0.49      0.50      0.49      5440

Test metrics
              precision    recall  f1-score   support

           A       0.41      0.47      0.44       789
           B       0.38      0.25      0.30       743
           C       0.48      0.56      0.52       788
           D       0.67      0.69      0.68       908

    accuracy                           0.50      3228
   macro avg       0.49      0.49      0.48      3228
weighted avg       0.49      0.50      0.49      3228



/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [212]:

ovr_model2 = OneVsRestClassifier(logReg)
ovr_model2.fit(train_x_smote_tomek, train_target_smote_tomek)
getPredictAndMetrics(ovr_model2, train_x_smote_tomek, train_target_smote_tomek, test_x_encoded, test_target)

Train metrics
              precision    recall  f1-score   support

           A       0.46      0.49      0.47      1010
           B       0.41      0.23      0.29      1038
           C       0.52      0.67      0.59      1089
           D       0.67      0.72      0.69      1085

    accuracy                           0.53      4222
   macro avg       0.51      0.53      0.51      4222
weighted avg       0.52      0.53      0.51      4222

Test metrics
              precision    recall  f1-score   support

           A       0.40      0.44      0.42       789
           B       0.36      0.22      0.28       743
           C       0.47      0.60      0.53       788
           D       0.68      0.68      0.68       908

    accuracy                           0.50      3228
   macro avg       0.48      0.49      0.48      3228
weighted avg       0.49      0.50      0.49      3228



Точність всіх моделей не висока, найкраща з них - це SVC на SMOTE даних model_svc_2 (судячи по F1 та recall ан тестових даних)
Найнижча точність моделей на класі В. Можливо варто викоритсовувати інший поріг входження для цього класу (якщо є така опція)